In [54]:
import numpy as np
import png
import os
import nibabel as nib
from matplotlib import pyplot as pl
import matplotlib
import math
import csv
import cv2
from tensorflow import keras
import tensorflow as tf
import sys
import statistics as stats

In [55]:
use_layer_selector = True

In [113]:
#sift = cv2.SIFT_create()
#bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)
#sample_img = cv2.imread('sample_brain.png')
#keypoints_0, descriptors_0 = sift.detectAndCompute(sample_img, None)

model = keras.models.load_model('Brain Layer Selector')

# input_folder = folder that contains the .nii file
# output_folder = folder that the .png file will be saved in
def nii_to_png(input_folder, output_folder, axis):
    filenames = os.listdir(input_folder)
    for f in filenames:
        if f.endswith('.nii'):
            #Start reading nii files
            img_path = os.path.join(input_folder, f)
            img = nib.load(img_path) #read nii            
            img_fdata = img.get_fdata()
            fname = filename[ filename.rfind('_')+1 : filename.find('.nii') ]
            
            input_shape = model.get_config()["layers"][0]["config"]["batch_input_shape"][1:3]
            (x,y,z) = img.shape
            if axis == 0:
                num_layers = x
            elif axis == 1:
                num_layers = y
            else:
                num_layers = z
            candidate_layers = []
            for i in range(int(0.25*num_layers), int(0.65*num_layers), 2): #x is the sequence of images
                if axis == 0:
                    slice = img_fdata[i, :, :] # cut perpendicular to the x axis
                elif axis == 1:
                    slice = img_fdata[:, i, :] # cut perpendicular to the y axis
                else:
                    slice = img_fdata[:, :, i] # cut perpendicular to the z axis
                    slice = np.rot90(slice)
                slice = cv2.resize(slice, dsize=input_shape) # set image size to 224x224
                slice = np.reshape(slice, (*slice.shape, 1)) # add an additional axis for concat
                slice /= np.max(slice)
                slice = np.concatenate((slice, slice, slice), axis=2) # concat 3 identical images (RGB channels)
                
                if use_layer_selector:
                    slice = slice[np.newaxis is None, :, :, :]
        
                    prediction = model.predict_step(slice).numpy()[0]
                    if prediction[0] > prediction[1]:
                        candidate_layers.append(slice)
                        
                else:
                    matplotlib.image.imsave(
                        os.path.join(output_folder,'{}-{}.png'.format(fname, i)), slice[0,:,:,:])
            
            if use_layer_selector:
                if len(candidate_layers) > 0:
                    layer = candidate_layers[ int(len(candidate_layers) / 2.5) ][0,:,:,:]
                else:
                    if axis == 0:
                        layer = img_fdata[int(0.33*x), :, :] # cut perpendicular to the x axis
                    elif axis == 1:
                        layer = img_fdata[:, int(0.33*y), :] # cut perpendicular to the y axis
                    else:
                        if fname == 'I275428':
                            layer = img_fdata[:, :, int(0.66*z)] # cut perpendicular to the z axis
                        else:
                            layer = img_fdata[:, :, int(0.33*z)] # cut perpendicular to the z axis
                        layer = np.rot90(layer)
                    layer = cv2.resize(layer, dsize=input_shape) # set image size to 224x224
                    layer = np.reshape(layer, (*layer.shape, 1)) # add an additional axis for concat
                    layer /= np.max(layer)
                    layer = np.concatenate((layer, layer, layer), axis=2) # concat 3 identical images (RGB channels)
                
                matplotlib.image.imsave(
                    os.path.join(output_folder,'{}.png'.format(fname)), layer)
                
            print(fname)

In [114]:
#for i in range(int(0.2*x), int(0.8*x), 2): #x is the sequence of images
#                slice = img_fdata[i, :, :] #Select which direction the slice can be
#                slice = np.reshape(slice, (*slice.shape, 1))
#                slice = np.concatenate((slice, slice, slice), axis=2)
                #slice = slice*255
                #slice = slice.astype('uint8')
                #keypoints_1, descriptors_1 = sift.detectAndCompute(slice, None)
                #matches = bf.match(descriptors_0, descriptors_1)
                #distance = 0
                #for m in matches:
                #    distance += m.distance
                
                #if min_sift_val > distance:
                #    layer_at_min = i
                #    min_sift_val = distance
#            matplotlib.image.imsave(
#                os.path.join(output_folder,'{}-{}.png'.format(fname, layer_at_min)),
#                img_fdata[layer_at_min, :, :])

In [120]:
input_folder = 'D:\\ADNI1_Complete 1Yr 1.5T'
output_folder = 'D:\\ADNI_PNG_separate'
metadata = 'D:\\ADNI1_Complete_1Yr_1.5T_1_26_2022.csv'

# use the csv metadata to correctly categorize the patients into CN, MCI, AD
img_group_dict = {}
# image groups: cognitively normal, mild cognitive impairment, Alzheimer's disease
group_folder_dict = {'CN':'NonDemented', 'MCI':'Demented', 'AD':'Demented'}
with open(metadata, newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', quotechar='\"')
    for row in reader:
        # this dictionary will be used to find what group (output folder) 
        # a patient belongs to
        img_group_dict[ row['Image Data ID'] ] = row['Group']

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# create subfolders in the output folder
for group in group_folder_dict:
    subfolder = os.path.join(output_folder, group_folder_dict[group])
    # put the generated subfolder directory in the dictionary for image export purposes
    group_folder_dict[group] = subfolder
    if not os.path.exists(subfolder):
        os.makedirs(subfolder)


In [121]:
#vgg19 = keras.models.load_model('Brain Layer Selector')
axes = ((),
        ('I134505', 'I133465', 'I133472', 'I133494', 'I133406', 'I133373', 'I133151', 'I133486'),
        ('I288891', 'I288892', 'I288900', 'I288901', 'I288902', 'I288917', 'I288918', 'I288924',
         'I288933', 'I288934', 'I296799', 'I296800', 'I296803', 'I296804', 'I299371', 'I299374',
         'I300285', 'I300286', 'I275427', 'I275428', 'I288893', 'I288894', 'I288903', 'I288904',
         'I296812', 'I296813', 'I312665', 'I312666'))

axis = 1

for root, dirnames, filenames in os.walk(input_folder):
    # no more subdirectories; we are looking at a folder with a .nii file
    img_id = root[ root.rfind('\\')+1 :]
    if len(dirnames) == 0 and (axis == 0 or img_id in axes[axis]):
        filename = filenames[0] # there's only one file in the folder so just pick it
        group = img_group_dict[img_id]
        #print(img_id, group, sep=', ')
        nii_to_png(root, group_folder_dict[group], axis)

I134505
